In [1]:
from talib import MACD, MACDEXT, RSI, BBANDS, MACD, AROON, STOCHF, ATR, OBV, ADOSC, MINUS_DI, PLUS_DI, ADX, EMA, SMA
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR, ROC
from talib import CDLABANDONEDBABY, CDL3BLACKCROWS,CDLDOJI, CDLDOJISTAR, CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR, CDLGRAVESTONEDOJI, CDLHAMMER, CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLINVERTEDHAMMER,CDLMARUBOZU,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLSPINNINGTOP,CDL3BLACKCROWS, CDL3LINESTRIKE, CDLKICKING

import pandas as pd
import numpy as np
import tables
import datetime as dt
import logging

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
import platform
from selenium import webdriver
import re
import os
from multiprocessing import Process
import gc
import warnings
import os
from multiprocessing import Process

logging.basicConfig(filename="log/live_log.log", filemode="a", level=logging.DEBUG, format="[%(asctime)s: %(levelname)s]:%(message)s")
logger=logging.getLogger()

KiteAPIKey = "b2w0sfnr1zr92nxm"
KiteAPISecret = "jtga2mp2e5fn29h8w0pe2kb722g3dh1q"


In [ ]:
nifty50 = pd.read_csv("data/ind_nifty50list.csv")
niftynext50 = pd.read_csv("data/ind_niftynext50list.csv")
midcap50 = pd.read_csv("data/ind_niftymidcap50list.csv")

downloadlist = nifty50['Symbol']
industry = niftynext50['Industry'].unique()

# Indicators and plots

In [6]:
fig = ""
noofcandles = 60
def candlestick(price, pos=1, plot=False):
    #print(price.index)
    global fig
    # Candlestick
    trace = go.Candlestick(x=price.index.astype('str'), open=price['open'], high=price['high'], low=price['low'], close=price['close'], name="Candlestick", showlegend=False)

    if plot:
        fig.append_trace(trace, pos, 1)
        fig['layout']['yaxis'+str(pos)]['title']="Candlestick"
    return price

def macd(price, pos=1, plot=False):
    global fig
    #price['macd'], price['macdsignal'], price['macdhist'] = MACD(price.close, fastperiod=12, slowperiod=26, signalperiod=9)
    price['macd'], price['macdsignal'], price['macdhist'] = MACDEXT(price.close, fastperiod=12, slowperiod=26, signalperiod=9, fastmatype=1, slowmatype=1,signalmatype=1)
        
    # list of values for the Moving Average Type:  
    #0: SMA (simple)  
    #1: EMA (exponential)  
    #2: WMA (weighted)  
    #3: DEMA (double exponential)  
    #4: TEMA (triple exponential)  
    #5: TRIMA (triangular)  
    #6: KAMA (Kaufman adaptive)  
    #7: MAMA (Mesa adaptive)  
    #8: T3 (triple exponential T3)
    
    # MACD plots
    traceMACD = go.Scatter(x=price.index.astype('str'), y=price.macd, name='MACD', line=dict(color='black'),showlegend=False)
    traceMACDSignal = go.Scatter(x=price.index.astype('str'), y=price.macdsignal, name='MACD signal', line=dict(color='red'),showlegend=False)
    traceMACDHist = go.Bar(x=price.index.astype('str'), y=price.macdhist, name='MACD Hist', marker=dict(color="grey"),showlegend=False)
        
    if plot:
        fig.append_trace(traceMACD, pos, 1)
        fig.append_trace(traceMACDSignal, pos, 1)
        fig.append_trace(traceMACDHist, pos, 1)
        fig['layout']['yaxis'+str(pos)]['anchor']="x"
        fig['layout']['yaxis'+str(pos)]['side']="right"
        fig['layout']['yaxis'+str(pos)]['title']="MACD"
    
    return price

def rsi(price, pos=1, plot=False):
    global fig
    price['RSI'] = RSI(price.close, timeperiod=14)
    
    traceRSI = go.Scatter(x=price.index.astype('str'), y=price['RSI'],mode='lines', line=dict(color='rgb(63, 72, 204)'), name='RSI',showlegend=False)
    
    if plot:
        fig.append_trace(traceRSI, pos, 1)
        fig['layout']['yaxis'+str(pos)]['anchor']="x"
        fig['layout']['yaxis'+str(pos)]['tickvals']=[20,30,70,80]
        fig['layout']['yaxis'+str(pos)]['side']="right"
        fig['layout']['yaxis'+str(pos)]['title']="RSI"
        
    return price

def stoch(price, pos=1, plot=False):
    global fig
    price['slowk'], price['slowd'] = STOCHF(price.high, price.low, price.close)
    
    traceSK  = go.Scatter(x=price.index.astype('str'), y=price['slowk'], name='%K', line=dict(color='black'), yaxis='y3',showlegend=False)
    traceSD  = go.Scatter(x=price.index.astype('str'), y=price['slowd'], name='%D', line=dict(color='red'),showlegend=False)
    
    if plot:
        fig.append_trace(traceSK, pos, 1)
        fig.append_trace(traceSD, pos, 1)
        fig['layout']['yaxis'+str(pos)]['title']="Stochastics"
        fig['layout']['yaxis'+str(pos)]['side']="right"
        fig['layout']['yaxis'+str(pos)]['anchor']="x"
        fig['layout']['yaxis'+str(pos)]['tickvals']=[20,80,30,70]
    return price

def aroon(price, pos=1, plot=False):
    global fig
    price['aroondown'], price['aroonup'] = AROON(price.high, price.low, 25)
    
    traceARD  = go.Scatter(x=price.index.astype('str'), y=price['aroondown'], name='AROON Down', line=dict(color='red'), yaxis='y4',showlegend=False)
    traceARU  = go.Scatter(x=price.index.astype('str'), y=price['aroonup'], name='AROON UP', line=dict(color='green'),showlegend=False)
    
    if plot:
        fig.append_trace(traceARD, pos, 1)
        fig.append_trace(traceARU, pos, 1)
        fig['layout']['yaxis'+str(pos)]['side']="right"
        fig['layout']['yaxis'+str(pos)]['title']="Aroon"
        fig['layout']['yaxis'+str(pos)]['anchor']="x"
    return price

def atr(price, pos=1, plot=False, multiplier=2):
    global fig
    price['atr'] = ATR(price.high, price.low, price.close, timeperiod=22)
    price['chandlierLong'] = price.close-multiplier*price.atr
    price['chandlierShort'] = price.close+multiplier*price.atr
    
    traceATR = go.Scatter(x=price.index.astype('str'), y=price['atr'],mode='lines', line=dict(color='lightgrey'), name='ATR',showlegend=False)
    
    traceStopLoss = go.Scatter(x=price.index, y=price['chandlierLong'].shift(1), mode='lines', line=dict(color='lightgrey'), name='SL',showlegend=False)
    if plot:
        fig.append_trace(traceStopLoss, pos, 1)
        
    return price

def bbands(price, pos=1, plot=False, plotPrice=False):
    global fig
    price['bbt'], price['bbm'], price['bbb'] = BBANDS(price.close, timeperiod=20, nbdevup=1.85, nbdevdn=1.85, matype=0)
    price['bbt2'], price['bbm2'], price['bbb2'] = BBANDS(price.close, timeperiod=20, nbdevup=2.15, nbdevdn=2.15, matype=0)
    
    tracePrice = go.Scatter(x=price.index.astype('str'), y=price.close, marker = dict(color='grey', size=2), mode='lines', name="Close Price", yaxis='y1', showlegend=False)
    traceBBT = go.Scatter(x=price.index.astype('str'), y=price['bbt'], name='BB_up',  line=dict(color='lightgrey'),showlegend=False)
    traceBBB = go.Scatter(x=price.index.astype('str'), y=price['bbb'], name='BB_low',  line=dict(color='lightgrey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False)
    traceBBM = go.Scatter(x=price.index.astype('str'), y=price['bbm'], name='BB_mid',  line=dict(color='lightgrey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False)
    
    traceBBT2 = go.Scatter(x=price.index.astype('str'), y=price['bbt2'], name='BB_up2',  line=dict(color='blue'),showlegend=False)
    traceBBB2 = go.Scatter(x=price.index.astype('str'), y=price['bbb2'], name='BB_low2',  line=dict(color='blue'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False)
    #traceBBM2 = go.Scatter(x=price.index.astype('str'), y=price['bbm2'], name='BB_mid2',  line=dict(color='grey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False)
    
    
    if plot:
        if plotPrice:
            fig.append_trace(tracePrice, pos, 1)
            
        fig.append_trace(traceBBT, pos, 1)
        fig.append_trace(traceBBB, pos, 1)
        fig.append_trace(traceBBM, pos, 1)
        
        fig.append_trace(traceBBT2, pos, 1)
        fig.append_trace(traceBBB2, pos, 1)
        #fig.append_trace(traceBBM2, pos, 1)
    
    return price

def sma(price, pos=1, plot=False, plotPrice=False):
    global fig
    price['sma20'] = SMA(price.close, timeperiod=50)
    price['sma50'] = SMA(price.close, timeperiod=200)
    
    tracePrice = go.Scatter(x=price.index.astype('str'), y=price.close, marker = dict(color='grey', size=2), mode='lines', name="Close Price", yaxis='y1', showlegend=False)
    traceSMA20 = go.Scatter(x=price.index.astype('str'), y=price['sma20'], name='SMA20',  line=dict(color='black'),showlegend=False)
    traceSMA50 = go.Scatter(x=price.index.astype('str'), y=price['sma50'], name='SMA50',  line=dict(color='red'),showlegend=False)
    
    
    if plot:
        if plotPrice:
            fig.append_trace(tracePrice, pos, 1)
            
        fig.append_trace(traceSMA20, pos, 1)
        fig.append_trace(traceSMA50, pos, 1)
    
    return price

def obv(price, pos=1, plot=False):
    global fig
    price['obv'] = OBV(np.linspace(1,100,price.index.shape[0]), price.volume)
    
    traceV = go.Bar(x=price.index.astype('str'), y=price.volume, name='Volume', marker=dict(color='blue'), showlegend=False)
    traceOBV = go.Scatter(x=price.index.astype('str'), y=price['obv'], name='OBV', line=dict(color='black'),showlegend=False)
    if plot:
        fig.append_trace(traceV, pos, 1)
        
        #fig['layout']['yaxis2']['overlaying']='y'
        fig['layout']['yaxis'+str(pos)]['anchor']='x'
        fig['layout']['yaxis'+str(pos)]['showgrid']=True
        fig['layout']['yaxis'+str(pos)]['visible']=True
        
        maxV = round(price.iloc[-3*noofcandles:-1]['volume'].max()/2,0)*2
        fig['layout']['yaxis'+str(pos)]['tickvals']=[maxV/2,maxV]
        fig['layout']['yaxis'+str(pos)]['range']=[0,maxV*2]
        fig['layout']['yaxis'+str(pos)]['side']="right"
        fig['layout']['yaxis'+str(pos)]['title']="volume"
    return price

def adosc(price, pos=1, plot=False):
    global fig
    
    price['adosc'] = ADOSC(price.high, price.low, price.close, price.volume)
    return price

def adx(price, pos=1, plot=False):
    global fig
    price['ADX'] = ADX(price.high, price.low, price.close)
    price['DI_P'] = PLUS_DI(price.high, price.low, price.close)
    price['DI_M'] = MINUS_DI(price.high, price.low, price.close)
    
    traceADX  = go.Scatter(x=price.index.astype('str'), y=price['ADX'], name='ADX', line=dict(color='blue') ,showlegend=False)
    traceDIP  = go.Scatter(x=price.index.astype('str'), y=price['DI_P'], name='DI+', line=dict(color='green'),showlegend=False)
    traceDIM  = go.Scatter(x=price.index.astype('str'), y=price['DI_M'], name='DI-', line=dict(color='red'),showlegend=False)
    
    
    if plot:
        fig.append_trace(traceADX, pos, 1)
        fig.append_trace(traceDIP, pos, 1)
        fig.append_trace(traceDIM, pos, 1)
        fig['layout']['yaxis'+str(pos)]['title']="ADX"
        fig['layout']['yaxis'+str(pos)]['side']="right"
        fig['layout']['yaxis'+str(pos)]['anchor']="x"
    return price

# Heikin-Ashi
def ha(price, pos=1, plot=False):
    global fig
    price['HAopen'] = (price.open.shift(1) + price.close.shift(1))/2
    price['HAhigh'] = price[['high','open','close']].max(axis = 1, skipna = True)
    price['HAlow'] = price[['low','open','close']].min(axis = 1, skipna = True)
    price['HAclose'] = (price.open+price.high+price.low+price.close)/4
    
    traceHA = go.Candlestick(x=price.index.astype('str'), open=price['HAopen'], high=price['HAhigh'], low=price['HAlow'], close=price['HAclose'], name="Heikin Ashi", showlegend=False)
    
    if plot:
        fig.append_trace(traceHA, pos, 1)
    return price

def pivotPoint(price, pos=1,fibo=True, noofcandles=60, plot=False):
    global fig
    prev = raw_data_day.loc[prevday]

    PP = (prev.high+prev.low+prev.close)/3
    S1 = PP - 0.382 * (prev.high - prev.low)
    S2 = PP - 0.618 * (prev.high - prev.low)
    S3 = PP -         (prev.high - prev.low)

    R1 = PP + 0.382 * (prev.high - prev.low)
    R2 = PP + 0.618 * (prev.high - prev.low)
    R3 = PP +         (prev.high - prev.low)
    
    tracePP  = go.Scatter(x=price.index.astype('str'), y=np.full(raw_data.index.shape[0],PP), line=dict(color='red'),showlegend=False)
    traceS1  = go.Scatter(x=price.index.astype('str'), y=np.full(raw_data.index.shape[0],S1), line=dict(color='red'),showlegend=False)
    traceS2  = go.Scatter(x=price.index.astype('str'), y=np.full(raw_data.index.shape[0],S2), line=dict(color='red'),showlegend=False)
    traceS3  = go.Scatter(x=price.index.astype('str'), y=np.full(raw_data.index.shape[0],S3), line=dict(color='red'),showlegend=False)
    traceR1  = go.Scatter(x=price.index.astype('str'), y=np.full(raw_data.index.shape[0],R1), line=dict(color='red'),showlegend=False)
    traceR2  = go.Scatter(x=price.index.astype('str'), y=np.full(raw_data.index.shape[0],R2), line=dict(color='red'),showlegend=False)
    traceR3  = go.Scatter(x=price.index.astype('str'), y=np.full(raw_data.index.shape[0],R3), line=dict(color='red'),showlegend=False)
    
    
    
    if plot:
        fig.append_trace(tracePP, pos, 1)
        fig.append_trace(traceS1, pos, 1)
        fig.append_trace(traceS2, pos, 1)
        fig.append_trace(traceS3, pos, 1)
        fig.append_trace(traceR1, pos, 1)
        fig.append_trace(traceR2, pos, 1)
        fig.append_trace(traceR3, pos, 1)
    return price

def emasma(price, plot=False):
    global fig
    price['ema21'] = EMA(price['close'], 21)
    price['ema9'] = EMA(price['close'], 9)
    price['sma200'] = SMA(price['close'], 200)
    price['sma50'] = SMA(price['close'], 50)
    
    return price

def detectCDPattern(prices, plot=True, noofcandles=15, strPlot="engulfing"):
    global fig
    #https://github.com/mrjbq7/ta-lib
    
    annotateText = []
    annotateIndex = []
    
    for index, curr in prices.iloc[noofcandles:].iterrows():
        text = ""
        
        index_r = raw_data.index.get_loc(index)+1
        index_l = max(0, index_r - noofcandles)
        #rice = pd.DataFrame()
        popen = prices.iloc[index_l:index_r]['open']
        phigh = prices.iloc[index_l:index_r]['high']
        pclose = prices.iloc[index_l:index_r]['close']
        plow = prices.iloc[index_l:index_r]['low']
        
        
        if (CDLDOJI(popen, phigh, plow, pclose)[-1] != 0) and (strPlot.find("Doji") >= 0):
            text += "Doji,"
        
        if CDLENGULFING(popen, phigh, plow, pclose)[-1] == -100 and strPlot.find("engulfing")>=0:
            text += "Bearish Engulfing,"
            
        if CDLENGULFING(popen, phigh, plow, pclose)[-1] == 100 and strPlot.find("engulfing")>=0:
            #print(CDLENGULFING(popen, phigh, plow, pclose))
            #print(index.strftime("%d-%m-%y %H:%M"))
            text += "Bullish Engulfing,"
            
        
        if CDLABANDONEDBABY(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("abandonedbaby")>=0:
            text += "Abandoned Baby,"
    
        if CDL3BLACKCROWS(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("3blackcrows")>=0:
            text += "3 Black Crows,"
    
    
        if CDLDOJISTAR(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("dojistar")>=0:
            text += "Doji Star,"
    
        if CDLDRAGONFLYDOJI(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("dragonflydoji")>=0:
            text += "Dragon Fly Doji,"
    
        if CDLEVENINGDOJISTAR(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("eveningdojistar")>=0:
            text += "Evening Doji Star,"
    
        if CDLEVENINGSTAR(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("eveningstar")>=0:
            text += "Evening Star,"
    
        if CDLGRAVESTONEDOJI(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("gravestonedoji")>=0:
            text += "Gravestone Doji,"
    
        if CDLHAMMER(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("hammer")>=0:
            text += "Hammer,"
    
        if CDLHANGINGMAN(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("hangingman")>=0:
            text += "Hanging Man,"
    
        if CDLHARAMI(popen, phigh, plow, pclose)[-1] == 100 and strPlot.find("Harami")>=0:
            text += "Bullish Harami,"
            
        if CDLHARAMI(popen, phigh, plow, pclose)[-1] == -100 and strPlot.find("Harami")>=0:
            text += "Bearish Harami,"
    
        if CDLHARAMICROSS(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("haramicross")>=0:
            text += "Harami Cross,"
    
        if CDLINVERTEDHAMMER(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("invertedhammer")>=0:
            text += "Inverted Hammer,"
    
        if CDLMARUBOZU(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("marubozu")>=0:
            text += "Marubozu,"
    
        if CDLMORNINGDOJISTAR(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("morningdojistar")>=0:
            text += "Morning Doji Star,"
    
        if CDLMORNINGSTAR(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("morningstar")>=0:
            text += "Morning Star,"
    
        if CDLSHOOTINGSTAR(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("shootingstar")>=0:
            text += "Shooting Star,"
    
    
        if CDLSPINNINGTOP(popen, phigh, plow, pclose)[-1] and strPlot.find("spinningtop")>=0:
            text += "Spinning Top,"
    
        if CDL3LINESTRIKE(popen, phigh, plow, pclose)[-1] and strPlot.find("3linestrike")>=0:
            text += "3 Line Strike,"
        
        
        if text != "":
            annotateText.append(text)
            annotateIndex.append(index)
            
            #print(index.strftime("%d-%m-%y %H:%M")+": "+text)
    

    data = pd.DataFrame({'open':prices.loc[annotateIndex, 'open'],
                         'close':prices.loc[annotateIndex, 'close'],
                         'high':prices.loc[annotateIndex, 'high'],
                         'low':prices.loc[annotateIndex, 'low'],
                         'text':annotateText})
    prices['text'] =""
    
    prices['text'] = data['text']
    
    if plot:
        annotateBuySell(data)
    return prices

def calculateStats(price):
    global fig
    price['priceSD'] = STDDEV(price.close)
    price['priceVAR'] = VAR(price.close)

    price['delBBT'] = (price.bbt - price.close)/(price.bbt-price.bbm)
    price['delBBB'] = (price.close - price.bbb)/(price.bbm-price.bbb)

    price['priceTSF'] = TSF(price.close)

    price['priceROC'] = ROC(price.close)
    price['macdROC'] = ROC(price.macd)
    price['macdsigROC'] = ROC(price.macd)
    price['bbROC'] = ROC(price.bbm)

    price['sdPriceRange'] = STDDEV((price.high - price.close))
    price['priceRange'] = (price.high - price.close)

    return price


def annotateBuySell(price, AnnotateType=""):
    global fig
    arr = []
    for index, row in price.iterrows():
        if AnnotateType == "Buy":
            arr.append(dict(x=index,y=row['low']-1, xref='x',yref='y', ax=0, ay=35,
                                       showarrow=True, arrowhead=1,arrowsize=0.5, arrowwidth=7,arrowcolor='green'))
        elif AnnotateType == "Sell":
            arr.append(dict(x=index,y=row['high']+1, xref='x',yref='y', ax=0, ay=-35,
                                       showarrow=True, arrowhead=1,arrowsize=0.5, arrowwidth=7,arrowcolor='red'))
        else:
            arr.append(dict(x=index,y=row['low']-1, xref='x',yref='y', ax=0, ay=50, text=row["text"],
                                       showarrow=True, arrowhead=1,arrowsize=1, arrowwidth=1,arrowcolor='black'))
            
    
    arr.extend(fig['layout']['annotations'])
    fig['layout']['annotations']=arr
    return price

def createPlot(symbol):
    global fig
    fig = tools.make_subplots(rows=5, cols=1, shared_xaxes=True, row_width=[1,1,3,1,5], vertical_spacing = 0.01)
    fig['layout']['xaxis'] = dict(rangeslider = dict(visible=False), side="bottom") #, range=[xMin,xMax])
    fig['layout'].update(height=950, plot_bgcolor='rgba(0,0,0,0)', title="Charts for "+symbol)
    #fig['layout']['yaxis']['range'] = [yMin, yMax]
    fig['layout']['yaxis']['anchor'] = 'x'
    fig['layout']['yaxis']['side'] = 'right'
    
    fig['layout']['xaxis']['rangeselector'] = dict(
                buttons=list([dict(count=1, label='1h', step='hour', stepmode='backward'),
                              dict(count=3, label='3h', step='hour', stepmode='backward'),
                              dict(count=6, label='1d', step='hour', stepmode='backward'),
                              dict(step='all')]))
    return fig

def plotData(filename="analysis", auto_open=True, inline = True):
    global fig
    init_notebook_mode(connected=True)
    if inline:
        iplot(fig, filename="plot/"+filename+toDate.strftime("_%d_%m_%y")+".html")
    else:
        plot(fig, filename="plot/"+filename+toDate.strftime("_%d_%m_%y")+".html", auto_open=auto_open)

In [ ]:
def plot_set1(temp_data): 
    global fig, noofcandles
    noofcandles = 60
    yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
    yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()

    xMin = temp_data.index[-1*noofcandles]
    xMax = temp_data.index[-1]
    fig['layout']['yaxis']['range'] = [yMin, yMax]
    fig['layout']['yaxis']['range'] = [xMin, xMax]

    temp_data = candlestick(temp_data,1,True)
    temp_data = bbands(temp_data,1, True)

    temp_data = macd(temp_data,3,True)
    temp_data = rsi(temp_data,4, True)
    #temp_data = aroon(temp_data,5, True)
    temp_data = stoch(temp_data,5, True)
    temp_data = sma(temp_data,1,True)
    obv(temp_data,2, True)

# Historical Data Download

In [3]:
def getInstruments(exchange='NSE'):
    instruments_df = pd.DataFrame(data=kite.instruments(exchange))
    instruments_df = instruments_df.set_index('tradingsymbol')
    return instruments_df

def downloadData(symbol="HDFC", fromDate= dt.datetime.now() - dt.timedelta(days = 1), toDate=dt.datetime.now(), freq="minute"):
    symbolToken = instruments_df.loc[symbol,'instrument_token']
    raw_data = pd.DataFrame(data=kite.historical_data(symbolToken, fromDate, toDate, freq, continuous=False))
    raw_data = raw_data.set_index('date').tz_localize(None)
    return raw_data

def resample2(data,freq):
    data = data.resample(freq).agg({'open':'first','high':'max','low':'min','close':'last','volume':'sum'})
    #data.columns = data.columns.droplevel()
    return data

def getData(symbol, fromDate, toDate, exchange="NSE", freq="minute"):
    #symbol = "SBIN"
    key = freq+"/"+exchange+"/"+symbol

    #fromDate = dt.datetime(2019,4,8)
    #toDate = dt.datetime(2019,4,10)
    
    try:
        rDate = pd.HDFStore("kite_data/kite_cache.h5").get(key).tail(1).index
        lDate = pd.HDFStore("kite_data/kite_cache.h5").get(key).head(1).index
        
        #print(fromDate,toDate, lDate, rDate)
        raw_data = pd.read_hdf("kite_data/kite_cache.h5", key=key, mode="a", format="table")

        if   (fromDate < lDate ) and (toDate <= rDate):
            logging.debug("Downloading data from fromDate to lDate")
            temp_data = downloadData(symbol,  fromDate, lDate, freq)
            temp_data = temp_data.append(raw_data.tail(-1))
#            temp_data.to_hdf("kite_data/kite_cache.h5", key=key, mode="a", format="table")
        elif (fromDate >=lDate ) and (toDate <= rDate):
            logging.debug("Using cache: Not downloading data")
            temp_data = raw_data
        elif (fromDate >= lDate ) and (toDate > rDate):
            logging.debug("Downloading data from rDate to toDate")
            temp_data = downloadData(symbol,  rDate, toDate, freq)
            temp_data = raw_data.append(temp_data.tail(-1))
#            temp_data.to_hdf("kite_data/kite_cache.h5", key=key, mode="a", format="table")
        elif (fromDate < lDate ) and (toDate > rDate):
            logging.debug("Downloading data from fromDate to lDate")
            temp_data = downloadData(symbol,  fromDate, lDate, freq)
            temp_data = temp_data.append(raw_data.tail(-1))
            logging.debug("Downloading data from rDate to toDate")
            temp_data2 = downloadData(symbol,  rDate, toDate, freq)
            temp_data = temp_data.append(temp_data2.tail(-1))
#            temp_data.to_hdf("kite_data/kite_cache.h5", key=key, mode="a", format="table")

    except Exception as e:
        logging.debug(e)
        temp_data = downloadData(symbol, fromDate, toDate, freq)
    finally:
        temp_data.to_hdf("kite_data/kite_cache.h5", key=key, mode="a", format="table")
        return temp_data[(temp_data.index >= fromDate) & (temp_data.index <= toDate)]
    
def portfolioDownload(stocklist, toDate):
    stocklist_df = pd.DataFrame()
    for index, row in stocklist.iterrows():
        symbol = row[0]
        logging.debug("Downloading data for: "+symbol)
        temp_data = getData(symbol,  toDate - dt.timedelta(days = 5), toDate)
        temp_data['symbol'] = symbol
        temp_data.set_index(['symbol',temp_data.index], inplace=True)
        #print(temp_data)
        stocklist_df = stocklist_df.append(temp_data)
    
    #print(stocklist_df)
    return stocklist_df

In [3]:
#help(MACD)

Help on function MACD in module talib._ta_lib:

MACD(...)
    MACD(real[, fastperiod=?, slowperiod=?, signalperiod=?])
    
    Moving Average Convergence/Divergence (Momentum Indicators)
    
    Inputs:
        real: (any ndarray)
    Parameters:
        fastperiod: 12
        slowperiod: 26
        signalperiod: 9
    Outputs:
        macd
        macdsignal
        macdhist



# Kite Authentication and wrappers

In [2]:
kite = KiteConnect(api_key=KiteAPIKey)
reauthentication = False

f = open("kite_data/access_token.txt", mode="r")
access_token = f.readlines()
logger.info(access_token[0])
kite.set_access_token(access_token[0])

try:
    logger.info("Welcome "+kite.profile()['user_name'])
except:
    if platform.system() == "Windows":
        driver = webdriver.Chrome("./automation/chromedriver.exe")
    else:
        driver = webdriver.Chrome("./automation/chromedriver")
    driver.implicitly_wait(60) ;
    driver.get(kite.login_url())
    reauthentication = True
    

In [3]:
if reauthentication:
    try:
        request_token = re.findall("request_token=(.*)&action",driver.current_url)
    except:
        request_token = re.findall("request_token=(.*)",driver.current_url)

    data = kite.generate_session(request_token[0], api_secret=KiteAPISecret)
    access_token = data["access_token"]
    kite.set_access_token(access_token)
    f = open("kite_data/access_token.txt", mode="w")
    f.write(access_token)
    f.close()
    driver.close()

In [ ]:
try:
    if exchange=="":
        exchange = "NSE"
except:
    logging.debug("Exchange not defined: Using default NSE")
    exchange = "NSE"

try:
    instruments_df = getInstruments(exchange)
    instruments_df.to_hdf('kite_data/kite_cache.h5', key=exchange, mode='a', format="table")
except:
    instruments_df = pd.read_hdf('kite_data/kite_cache.h5', key=exchange, mode='r', format="table")

EQSYMBOL = lambda x:instruments_df[instruments_df['instrument_token']==x].index[0]
EQTOKEN = lambda x:instruments_df.loc[x,'instrument_token']

In [ ]:
#logging.critical("BUY"+symbol)
def buy_slm(symbol, price, trigger,quantity=1): 
    logging.critical("BUY: "+symbol+", price: "+str(price)+", stoploss: "+str(stoploss)+", quantity: "+str(quantity))
    
    if papertrade:
        return
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_BUY,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_SLM,
                                product=kite.PRODUCT_MIS,
                                trigger_price=round(trigger,1),
                                #stoploss=round(stoploss,1),
                                #price=price,
                                variety=kite.VARIETY_REGULAR
                                )
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))
        
def buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1): 
    logging.critical("BUY: "+symbol+", price: "+str(price)+", squareoff: "+str(squareoff)+", stoploss: "+str(stoploss)+", quantity: "+str(quantity))
    if papertrade:
        return
    
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_BUY,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_SL,
                                product=kite.PRODUCT_MIS,
                                stoploss=stoploss,
                                trigger_price=trigger,
                                price=price,
                                squareoff=squareoff,
                                variety=kite.VARIETY_BO
                                )
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))


def sell_slm(symbol, price, trigger, quantity=1):
    
    logging.critical("SELL: "+symbol+", price: "+str(price)+", stoploss: "+str(stoploss)+", quantity: "+str(quantity))
       
    if papertrade:
         return
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                            exchange=kite.EXCHANGE_NSE,
                            transaction_type=kite.TRANSACTION_TYPE_SELL,
                            quantity=quantity,
                            order_type=kite.ORDER_TYPE_SLM,
                            product=kite.PRODUCT_MIS,
                            trigger_price=round(trigger,1),
                            #price=price,
                            variety=kite.VARIETY_REGULAR)
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))

def sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1): 
    logging.critical("SELL: "+symbol+", price: "+str(price)+", squareoff: "+str(squareoff)+", stoploss: "+str(stoploss)+", quantity: "+str(quantity))
    if papertrade:
        return
    
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_SELL,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_SL,
                                product=kite.PRODUCT_MIS,
                                stoploss=stoploss,
                                trigger_price=trigger,
                                price=price,
                                squareoff=squareoff,
                                variety=kite.VARIETY_BO
                                )
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))
        
def getOrders():    
    # Fetch all orders
    return pd.DataFrame(kite.orders())

def cancelOrder(orderId):
    if papertrade:
        logging.critical("In Paper Trade Mode: Order cancellation not possible")
        return
    
    try:
        kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=orderId, parent_order_id=None)    
    except Exception as e:
        logging.info("Order Cancellation failed: {}".format(e.message))

In [ ]:
def resample(ws, freq="1min"):
    #F = open("kite_data/recommendation.csv","a") 
    
    logging.debug(str(ws.prevtimeStamp)+": In resampler function")
    
    if ws.LiveStream.empty:
        logging.debug(str(ws.prevtimeStamp)+": Empty dataframe, Exiting resampler")
        return
      
    LiveStream2 = ws.LiveStream.groupby(['symbol','date']).agg({'price':['first','max','min','last'], 'volume':['last']})
    LiveStream2.columns = LiveStream2.columns.droplevel()
    LiveStream2.columns = ['open', 'high','low','close', 'volume']

    for index, data in LiveStream2.groupby(level=0):
        sampled = data.loc[index].resample(freq).agg({'open':{'open':'first'},'high':{'high':'max'},'low':{'low':'min'},'close':{'close':'last'},'volume':{'volume':'last'}})
        sampled.columns = sampled.columns.droplevel()

        sampled['symbol'] = index
        sampled.set_index(['symbol',sampled.index], inplace=True)

        ws.LiveStreamOHLC = ws.LiveStreamOHLC.append(sampled)
        
    #logger.info(index+":\n"+str(ws.LiveStreamOHLC.loc[portfolio.iloc[-1,0]].tail(240)))

    ws.LiveStreamOHLC.to_csv("kite_data/livestreamohlc.csv", mode='a')

    a = algoTrade(ws.LiveStreamOHLC.loc[portfolio.iloc[-1,0]].tail(120))
    #reco_df = a.tradeRecommendation()
    a.tradeBot(portfolio.iloc[-1,0])
    
    #try:
    #    if reco_df != "":
    #        F.write(reco_df)
    #except   Exception as e:
    #    logging.critical("Not able to resample and analyse data"+str(e))
    #finally:
    #    F.close()
        



def ticksHandler(ws, ticks):
    timeStamp = dt.datetime.now().replace(second=0, microsecond=0)
    tick_df = pd.DataFrame(ticks)
    
    #logging.debug("Prev: "+str(ws.prevtimeStamp)+"\nNow: "+str(timeStamp))
    
    try:
        tick_df.loc[tick_df['timestamp'].isna(), 'timestamp'] = timeStamp
        tick_df = tick_df[['timestamp','instrument_token','last_price','volume']]
        tick_df.instrument_token = tick_df.instrument_token.apply(EQSYMBOL)
        tick_df.columns = ['date','symbol','price','volume']
        tick_df.set_index(['symbol','date'], inplace=True)
        
        timeStamp = tick_df.index[0][-1].to_pydatetime()
        
        #logging.info(str(timeStamp - ws.prevtimeStamp))
        #print(type(timeStamp))
        
        #logging.debug("Exception: ticksHandler: "+str(tick_df))
        #timeStamp = tick_df.loc[portfolio.iloc[-1,0]].index[-1]
    except  Exception as e:
        logging.debug("Exception: ticksHandler: "+str(e)+str(tick_df))
        
    if( (timeStamp - ws.prevtimeStamp) >= dt.timedelta(minutes=5)):
        ws.prevtimeStamp = timeStamp
        
        #ws.cbufferSampling = ws.cbufferAccum.copy(deep=True) #Copy accumulated stream to sampling buffer
        #del ws.cbufferAccum
        #gc.collect()
        #ws.cbufferAccum = pd.DataFrame() #Reset accumulation buffer
        #logging.debug(ws.cbufferSampling)
        resample(ws, '5min')
    
    #ws.cbufferAccum = ws.cbufferAccum.append(tick_df)
    
    ws.LiveStream = ws.LiveStream.append(tick_df)
    #ws.LiveStream.to_csv("kite_data/livestream.csv", mode='a')

def orderNotification(ws,data):
    logger.debug(data)

def initTrade(ws):
    ws.prevtimeStamp = dt.datetime.now() - dt.timedelta(minutes=10)
    toDate = dt.datetime.now()
    #ws.cbufferAccum = pd.DataFrame()
    #ws.cbufferSampling = pd.DataFrame()
    ws.LiveStream = pd.DataFrame()
    ws.LiveStreamOHLC = pd.DataFrame()
    ws.LiveStreamOHLC = portfolioDownload(portfolio, toDate)
    

In [ ]:
def on_ticks(ws, ticks):
    # Callback to receive ticks.
    #logging.debug("Ticks: {}".format(ticks))
    ticksHandler(ws, ticks)


def on_connect(ws, response):
    initTrade(ws)
    logger.debug(portfolioToken)
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    #ws.subscribe(portfolioToken)

    ws.subscribe(portfolioToken)
    
    # Set RELIANCE to tick in `full` mode.
    # MODE_LTP, MODE_QUOTE, or MODE_FULL

    ws.set_mode(ws.MODE_FULL, portfolioToken)
    #ws.set_mode(ws.MODE_FULL, [225537]) 
    #ws.set_mode(ws.MODE_LTP, [225537, 3861249]) 
    #ws.set_mode(ws.MODE_MODE_QUOTE, [2714625,779521]) 

def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()

def on_order_update(ws, data):
    logger.info("New Order Update")
    orderNotification(ws,data)

In [ ]:
# Initialise
kws = KiteTicker(KiteAPIKey, kite.access_token)

# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_order_update = on_order_update

# Backtesting

In [1]:
def backtest(portfolio, fromDate, toDate):
    global STATE, buy_df, sell_df
    offset = 90
    for index,symbol in portfolio.iterrows():
        profit = 0
        STATE = "BUY"
        squareoff = 0
        stoploss = 0
        global fig
        noofcandles = 60
        x = symbol[0]
        logging.debug(str(fromDate) + "," + str(toDate))
        temp_data = getData(x, fromDate = fromDate, toDate=toDate, freq="minute")
        rowSize = temp_data.shape[0]
        
        for i in np.linspace(offset,rowSize-1, rowSize-offset -1, dtype="int64"):
            if(toDate.date() != temp_data.index[i].date()):
                continue
            a = algoTrade(temp_data.iloc[(i-offset):i])
            
            temp_profit = a.tradeBot(x, temp_data.index[i].to_pydatetime())
            
            profit = profit + temp_profit
            
            temp_df = temp_data.iloc[i]
            temp_df['symbol'] = x
            #print(temp_df)
            
            if temp_profit > 0:
                buy_df = buy_df.append(temp_df)
            elif temp_profit < 0:
                sell_df = sell_df.append(temp_df)
                
            
            #print(temp_data.iloc[(i-30):i])
        
        if STATE == "SELL":
            profit = profit + temp_data.iloc[-1].close

        print(x+": "+str(round(profit,2))+", ("+str(round(profit/temp_data.iloc[-1].close*100,4))+" %)")
        logging.debug(x+": "+str(round(profit,2))+", ("+str(round(profit/temp_data.iloc[-1].close*100,4))+" %)")    